In [58]:
import os
import glob
import pandas as pd

In [59]:
 df

,folder,id,rating,content
0,pos,11590,10,"Oh, the sixties. There were some interesting f..."
1,pos,9479,10,Tian's remake is no good at all. I only click ...
2,pos,676,8,'The Last Wave' is far more than the sum of it...
3,pos,3217,8,Kurt Russell is so believable and the action s...
4,pos,7840,8,BORN TO BOOGIE is a real 'find'--though a rock...
...,...,...,...,...
9995,neg,5365,3,"""Private Practice"" is being spun off the fairl..."
9996,neg,9509,1,"The storyplot was okay by itself, but the film..."
9997,neg,10059,4,I was expecting a documentary that focused on ...
9998,neg,7968,1,"We've all been there, sitting with some friend..."


In [60]:
import os
import glob
import pandas as pd

# Path to your main folder
main_path = "train"

data = []

# Loop over each subfolder
for subfolder in os.listdir(main_path):
    if subfolder not in ["pos", "neg"]:  # skip unsup
        continue
    
    folder_path = os.path.join(main_path, subfolder)
    
    # Get all .txt files inside the subfolder
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))
    
    for file in txt_files:
        filename = os.path.basename(file).replace(".txt", "")
        file_id, rating = filename.split("_")  # split into ID and rating
        
        with open(file, "r", encoding="utf-8") as f:
            text = f.read()
            
        data.append({
            "folder": subfolder,   # pos or neg
            "id": int(file_id),    # numeric ID
            "rating": int(rating), # rating
            "content": text
        })

# Convert to DataFrame
df = pd.DataFrame(data)

# Take only 5000 samples from each class
df_pos = df[df["folder"] == "pos"].sample(n=5000, random_state=42)
df_neg = df[df["folder"] == "neg"].sample(n=5000, random_state=42)

# Final dataset
df_final = pd.concat([df_pos, df_neg]).reset_index(drop=True)




In [61]:
df_final

,folder,id,rating,content
0,pos,11590,10,"Oh, the sixties. There were some interesting f..."
1,pos,9479,10,Tian's remake is no good at all. I only click ...
2,pos,676,8,'The Last Wave' is far more than the sum of it...
3,pos,3217,8,Kurt Russell is so believable and the action s...
4,pos,7840,8,BORN TO BOOGIE is a real 'find'--though a rock...
...,...,...,...,...
9995,neg,5365,3,"""Private Practice"" is being spun off the fairl..."
9996,neg,9509,1,"The storyplot was okay by itself, but the film..."
9997,neg,10059,4,I was expecting a documentary that focused on ...
9998,neg,7968,1,"We've all been there, sitting with some friend..."


In [62]:
# Convert text to lowercase
df_final["content"] = df_final["content"].str.lower()

print(df_final.head())


  folder     id  rating                                            content
0    pos  11590      10  oh, the sixties. there were some interesting f...
1    pos   9479      10  tian's remake is no good at all. i only click ...
2    pos    676       8  'the last wave' is far more than the sum of it...
3    pos   3217       8  kurt russell is so believable and the action s...
4    pos   7840       8  born to boogie is a real 'find'--though a rock...


In [63]:
import re

# Remove HTML tags like <br>, <br />, <p>, etc.
df_final["content"] = df_final["content"].apply(lambda x: re.sub(r'<.*?>', '', x))

print(df_final.head())


  folder     id  rating                                            content
0    pos  11590      10  oh, the sixties. there were some interesting f...
1    pos   9479      10  tian's remake is no good at all. i only click ...
2    pos    676       8  'the last wave' is far more than the sum of it...
3    pos   3217       8  kurt russell is so believable and the action s...
4    pos   7840       8  born to boogie is a real 'find'--though a rock...


In [64]:
# Function to remove URLs and emails
def clean_text(text):
    text = re.sub(r'http\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)           # Remove emails
    return text

# Apply cleaning to the 'content' column
df_final['content'] = df_final['content'].apply(clean_text)

In [65]:
df_final

,folder,id,rating,content
0,pos,11590,10,"oh, the sixties. there were some interesting f..."
1,pos,9479,10,tian's remake is no good at all. i only click ...
2,pos,676,8,'the last wave' is far more than the sum of it...
3,pos,3217,8,kurt russell is so believable and the action s...
4,pos,7840,8,born to boogie is a real 'find'--though a rock...
...,...,...,...,...
9995,neg,5365,3,"""private practice"" is being spun off the fairl..."
9996,neg,9509,1,"the storyplot was okay by itself, but the film..."
9997,neg,10059,4,i was expecting a documentary that focused on ...
9998,neg,7968,1,"we've all been there, sitting with some friend..."


In [66]:
import string

# Function to clean text
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)
    # Remove emails
    text = re.sub(r'\S+@\S+', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove emojis
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags
                               u"\U00002700-\U000027BF"  # dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Apply to the 'content' column
df_final['content'] = df_final['content'].apply(clean_text)


In [67]:
import pandas as pd
import re

# Load stopwords
stop_words = set([
    'a', 'about', 'above', 'after', 'again', 'against', 'all', 'am', 'an', 'and',
    'any', 'are', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being',
    'below', 'between', 'both', 'but', 'by', 'could', "couldn't", 'did', "didn't",
    'do', 'does', "doesn't", 'doing', "don't", 'down', 'during', 'each', 'few',
    'for', 'from', 'further', 'had', "hadn't", 'has', "hasn't", 'have', "haven't",
    'having', 'he', "he'd", "he'll", "he's", 'her', 'here', "here's", 'hers',
    'herself', 'him', 'himself', 'his', 'how', "how's", 'i', "i'd", "i'll", "i'm",
    "i've", 'if', 'in', 'into', 'is', "isn't", 'it', "it's", 'its', 'itself', 'let\'s',
    'me', 'more', 'most', "mustn't", 'my', 'myself', 'no', 'nor', 'not', 'of', 'off',
    'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out',
    'over', 'own', 'same', "shan't", 'she', "she'd", "she'll", "she's", 'should',
    "shouldn't", 'so', 'some', 'such', 'than', 'that', "that's", 'the', 'their',
    'theirs', 'them', 'themselves', 'then', 'there', "there's", 'these', 'they',
    "they'd", "they'll", "they're", "they've", 'this', 'those', 'through', 'to',
    'too', 'under', 'until', 'up', 'very', 'was', "wasn't", 'we', "we'd", "we'll",
    "we're", "we've", 'were', "weren't", 'what', "what's", 'when', "when's", 'where',
    "where's", 'which', 'while', 'who', "who's", 'whom', 'why', "why's", 'with',
    "won't", 'would', "wouldn't", 'you', "you'd", "you'll", "you're", "you've",
    'your', 'yours', 'yourself', 'yourselves'
])

# Function to remove stopwords in a string
def remove_stopwords_vectorized(text):
    return ' '.join([w for w in text.split() if w not in stop_words])

# Apply to the content column
df_final['content'] = df_final['content'].apply(remove_stopwords_vectorized)


In [68]:
df_final

,folder,id,rating,content
0,pos,11590,10,oh sixties interesting films movie goer now en...
1,pos,9479,10,tians remake good click remake documentary see...
2,pos,676,8,last wave far sum parts merely disaster film s...
3,pos,3217,8,kurt russell believable action nonstop takes t...
4,pos,7840,8,born boogie real findthough rock fan nearly th...
...,...,...,...,...
9995,neg,5365,3,private practice spun fairly successful well w...
9996,neg,9509,1,storyplot okay film felt bubbly fake also wors...
9997,neg,10059,4,expecting documentary focused tobacco industry...
9998,neg,7968,1,weve sitting friends watching bad movie laughi...


In [69]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [70]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')  # extended WordNet


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bbuser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\bbuser\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [71]:
from textblob import Word

# Simple lemmatization function
def lemmatize_text(text):
    return ' '.join([Word(word).lemmatize() for word in text.split()])

# Apply to your DataFrame
df_final['content'] = df_final['content'].apply(lemmatize_text)


In [44]:
df_final

,folder,id,rating,content
0,pos,11590,10,oh sixty interesting film movie goer now enjoy...
1,pos,9479,10,tians remake good click remake documentary see...
2,pos,676,8,last wave far sum part merely disaster film si...
3,pos,3217,8,kurt russell believable action nonstop take th...
4,pos,7840,8,born boogie real findthough rock fan nearly th...
...,...,...,...,...
9995,neg,5365,3,private practice spun fairly successful well w...
9996,neg,9509,1,storyplot okay film felt bubbly fake also wors...
9997,neg,10059,4,expecting documentary focused tobacco industry...
9998,neg,7968,1,weve sitting friend watching bad movie laughin...


In [45]:
# Keep only words with length > 2
def keep_meaningful_tokens(text):
    return ' '.join([word for word in text.split() if len(word) > 2])

df_final['content'] = df_final['content'].apply(keep_meaningful_tokens)


In [47]:
df

,folder,id,rating,content
0,neg,0,3,Story of a man who has unnatural feelings for ...
1,neg,10000,4,Airport '77 starts as a brand new luxury 747 p...
2,neg,10001,4,This film lacked something I couldn't put my f...
3,neg,10002,1,"Sorry everyone,,, I know this is supposed to b..."
4,neg,10003,1,When I was little my parents took me along to ...
...,...,...,...,...
24995,pos,9998,9,"Seeing as the vote average was pretty low, and..."
24996,pos,9999,8,"The plot had some wretched, unbelievable twist..."
24997,pos,999,10,I am amazed at how this movie(and most others ...
24998,pos,99,8,A Christmas Together actually came before my t...


In [80]:
import os
import glob
import pandas as pd

# Path to your main folder
main_path = "train"

data = []

# Loop over each subfolder
for subfolder in os.listdir(main_path):
    if subfolder not in ["pos", "neg"]:  # skip unsup
        continue
    
    folder_path = os.path.join(main_path, subfolder)
    
    # Get all .txt files inside the subfolder
    txt_files = glob.glob(os.path.join(folder_path, "*.txt"))
    
    for file in txt_files:
        filename = os.path.basename(file).replace(".txt", "")
        file_id, rating = filename.split("_")  # split into ID and rating
        
        with open(file, "r", encoding="utf-8") as f:
            text = f.read()
            
        data.append({
            "folder": subfolder,   # pos or neg
            "id": int(file_id),    # numeric ID
            "rating": int(rating), # rating
            "content": text
        })

# Convert to DataFrame
df = pd.DataFrame(data)

# Take only 5000 samples from each class
df_pos = df[df["folder"] == "pos"].sample(n=5000, random_state=42)
df_neg = df[df["folder"] == "neg"].sample(n=5000, random_state=42)

# Final dataset
df = pd.concat([df_pos, df_neg]).reset_index(drop=True)

df


,folder,id,rating,content
0,pos,11590,10,"Oh, the sixties. There were some interesting f..."
1,pos,9479,10,Tian's remake is no good at all. I only click ...
2,pos,676,8,'The Last Wave' is far more than the sum of it...
3,pos,3217,8,Kurt Russell is so believable and the action s...
4,pos,7840,8,BORN TO BOOGIE is a real 'find'--though a rock...
...,...,...,...,...
9995,neg,5365,3,"""Private Practice"" is being spun off the fairl..."
9996,neg,9509,1,"The storyplot was okay by itself, but the film..."
9997,neg,10059,4,I was expecting a documentary that focused on ...
9998,neg,7968,1,"We've all been there, sitting with some friend..."


In [79]:
# 1. Rename content column in df_original
df_original = df.rename(columns={'content': 'uncleaned_review'})

# 2. Rename content column in df_cleaned
df_final = df_final.rename(columns={'content': 'cleaned_review'})

# 3. Merge both DataFrames (assuming they are in the same order)
df_final = pd.concat([df.reset_index(drop=True),
                      df_final['cleaned_review'].reset_index(drop=True)], axis=1)

# Show first rows
print(df_final.head())
df_final

KeyError: 'cleaned_review'

In [78]:
# 1. Rename content column

# 2. Drop the duplicate 'cleaned_review' (keep the last one)
df_final = df_final.rename(columns={'content': 'cleaned_review1'})

# Check result
df_final

,folder,id,rating,cleaned_review1
0,pos,11590,10,oh sixty interesting film movie goer now enjoy...
1,pos,9479,10,tians remake good click remake documentary see...
2,pos,676,8,last wave far sum part merely disaster film si...
3,pos,3217,8,kurt russell believable action nonstop take th...
4,pos,7840,8,born boogie real findthough rock fan nearly th...
...,...,...,...,...
9995,neg,5365,3,private practice spun fairly successful well w...
9996,neg,9509,1,storyplot okay film felt bubbly fake also wors...
9997,neg,10059,4,expecting documentary focused tobacco industry...
9998,neg,7968,1,weve sitting friend watching bad movie laughin...


In [77]:
df_final = df_final.loc[:, ~df_final.columns.duplicated()]
df_final

,folder,id,rating,cleaned_review1
0,pos,11590,10,oh sixty interesting film movie goer now enjoy...
1,pos,9479,10,tians remake good click remake documentary see...
2,pos,676,8,last wave far sum part merely disaster film si...
3,pos,3217,8,kurt russell believable action nonstop take th...
4,pos,7840,8,born boogie real findthough rock fan nearly th...
...,...,...,...,...
9995,neg,5365,3,private practice spun fairly successful well w...
9996,neg,9509,1,storyplot okay film felt bubbly fake also wors...
9997,neg,10059,4,expecting documentary focused tobacco industry...
9998,neg,7968,1,weve sitting friend watching bad movie laughin...


In [76]:
df


,folder,id,rating,content
0,neg,0,3,Story of a man who has unnatural feelings for ...
1,neg,10000,4,Airport '77 starts as a brand new luxury 747 p...
2,neg,10001,4,This film lacked something I couldn't put my f...
3,neg,10002,1,"Sorry everyone,,, I know this is supposed to b..."
4,neg,10003,1,When I was little my parents took me along to ...
...,...,...,...,...
24995,pos,9998,9,"Seeing as the vote average was pretty low, and..."
24996,pos,9999,8,"The plot had some wretched, unbelievable twist..."
24997,pos,999,10,I am amazed at how this movie(and most others ...
24998,pos,99,8,A Christmas Together actually came before my t...


In [81]:
df_final

,folder,id,rating,cleaned_review1
0,pos,11590,10,oh sixty interesting film movie goer now enjoy...
1,pos,9479,10,tians remake good click remake documentary see...
2,pos,676,8,last wave far sum part merely disaster film si...
3,pos,3217,8,kurt russell believable action nonstop take th...
4,pos,7840,8,born boogie real findthough rock fan nearly th...
...,...,...,...,...
9995,neg,5365,3,private practice spun fairly successful well w...
9996,neg,9509,1,storyplot okay film felt bubbly fake also wors...
9997,neg,10059,4,expecting documentary focused tobacco industry...
9998,neg,7968,1,weve sitting friend watching bad movie laughin...


In [82]:
df

,folder,id,rating,content
0,pos,11590,10,"Oh, the sixties. There were some interesting f..."
1,pos,9479,10,Tian's remake is no good at all. I only click ...
2,pos,676,8,'The Last Wave' is far more than the sum of it...
3,pos,3217,8,Kurt Russell is so believable and the action s...
4,pos,7840,8,BORN TO BOOGIE is a real 'find'--though a rock...
...,...,...,...,...
9995,neg,5365,3,"""Private Practice"" is being spun off the fairl..."
9996,neg,9509,1,"The storyplot was okay by itself, but the film..."
9997,neg,10059,4,I was expecting a documentary that focused on ...
9998,neg,7968,1,"We've all been there, sitting with some friend..."


In [83]:
# Suppose df_clean has the cleaned column 'cleaned_review'
df_final['uncleaned_review'] = df['content'].values
df_final

,folder,id,rating,cleaned_review1,uncleaned_review
0,pos,11590,10,oh sixty interesting film movie goer now enjoy...,"Oh, the sixties. There were some interesting f..."
1,pos,9479,10,tians remake good click remake documentary see...,Tian's remake is no good at all. I only click ...
2,pos,676,8,last wave far sum part merely disaster film si...,'The Last Wave' is far more than the sum of it...
3,pos,3217,8,kurt russell believable action nonstop take th...,Kurt Russell is so believable and the action s...
4,pos,7840,8,born boogie real findthough rock fan nearly th...,BORN TO BOOGIE is a real 'find'--though a rock...
...,...,...,...,...,...
9995,neg,5365,3,private practice spun fairly successful well w...,"""Private Practice"" is being spun off the fairl..."
9996,neg,9509,1,storyplot okay film felt bubbly fake also wors...,"The storyplot was okay by itself, but the film..."
9997,neg,10059,4,expecting documentary focused tobacco industry...,I was expecting a documentary that focused on ...
9998,neg,7968,1,weve sitting friend watching bad movie laughin...,"We've all been there, sitting with some friend..."
